In [59]:
# Clone the Repository
!git clone https://github.com/wsklee/alignment-handbook.git
%cd alignment-handbook
!python -m pip install .

Cloning into 'alignment-handbook'...
remote: Enumerating objects: 1037, done.
remote: Counting objects: 100% (755/755), done.
remote: Compressing objects: 100% (423/423), done.
remote: Total 1037 (delta 514), reused 442 (delta 298), pack-reused 282 (from 1)
Receiving objects: 100% (1037/1037), 283.15 KiB | 1.31 MiB/s, done.
Resolving deltas: 100% (576/576), done.
/content/alignment-handbook/alignment-handbook/alignment-handbook/alignment-handbook
Processing /content/alignment-handbook/alignment-handbook/alignment-handbook/alignment-handbook
  Preparing metadata (setup.py) ... done
  Created wheel for alignment-handbook: filename=alignment_handbook-0.4.0.dev0-py3-none-any.whl size=27988 sha256=d0846de59ba697e37bb7a0f9f8e328ce4bc8ea3f15276a6ee4ada7284d82c101
  Stored in directory: /root/.cache/pip/wheels/45/50/ed/f014976ba6dfba87711f35e09e08cdb43f025eabddba41b542
Successfully built alignment-handbook
  Attempting uninstall: alignment-handbook
    Found existing installation: alignment-ha

In [60]:
import torch
import random
import numpy as np

# Set seed for reproducibility
seed_value = 42
torch.manual_seed(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)

# If GPU is used, set the seed for CUDA as well
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed_value)

In [61]:
# Load the IMDB dataset
from datasets import load_dataset
dataset = load_dataset("imdb")
train_data = dataset["train"]
test_data = dataset["test"]

In [62]:
test_data

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [63]:
X_test = test_data["text"]

In [64]:
y_true = test_data["label"]

## Create smaller train dataset for centroid calculation

In [65]:
from datasets import Dataset

# Filter 1000 positive and 1000 negative samples
small_positive = train_data.filter(lambda x: x['label'] == 1).shuffle(seed=42).select(range(1000))
small_negative = train_data.filter(lambda x: x['label'] == 0).shuffle(seed=42).select(range(1000))

# Combine into a smaller dataset
small_train_data = Dataset.from_dict({
    "text": small_positive["text"] + small_negative["text"],
    "label": small_positive["label"] + small_negative["label"]
})

In [66]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          pipeline,
                          logging)

## Load Custom class: DistilBertCLModel

In [67]:
from transformers import AutoModel
from alignment.models.distilbert_cl import DistilBertCLModel  # Custom class

model_name = "wsklee/distilbert-sentiment-imdb-sft-cft"
model = DistilBertCLModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model.eval()

config.json:   0%|          | 0.00/610 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

DistilBertCLModel(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1)

In [68]:
# Ensure that the model is moved to the appropriate device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

DistilBertCLModel(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1)

# Calculation of centroids

If there is existing centroid value use it, if not calculate it

In [69]:
from tqdm import tqdm
import torch

def calculate_centroids(model, train_data, batch_size=8, device='cuda'):
    model.to(device)
    model.eval()

    positive_centroid = torch.zeros(768, device=device)
    negative_centroid = torch.zeros(768, device=device)
    positive_count = 0
    negative_count = 0

    positive_sentences = [ex['text'] for ex in train_data if ex['label'] == 1]
    negative_sentences = [ex['text'] for ex in train_data if ex['label'] == 0]

    with torch.no_grad():
        # Process positive sentences
        for i in tqdm(range(0, len(positive_sentences), batch_size), desc="Processing positive batches"):
            batch_sentences = positive_sentences[i:i + batch_size]
            positive_encodings = tokenizer(batch_sentences, truncation=True, padding=True, return_tensors="pt")

            input_ids = positive_encodings['input_ids'].to(device)
            attention_mask = positive_encodings['attention_mask'].to(device)

            # Get embeddings and accumulate them
            embeddings = model.get_embedding(input_ids, attention_mask)
            positive_centroid += embeddings.sum(dim=0)
            positive_count += embeddings.size(0)

            # Move variables to CPU and clear cache
            del input_ids, attention_mask, embeddings
            torch.cuda.empty_cache()

        # Process negative sentences
        for i in tqdm(range(0, len(negative_sentences), batch_size), desc="Processing negative batches"):
            batch_sentences = negative_sentences[i:i + batch_size]
            negative_encodings = tokenizer(batch_sentences, truncation=True, padding=True, return_tensors="pt")

            input_ids = negative_encodings['input_ids'].to(device)
            attention_mask = negative_encodings['attention_mask'].to(device)

            # Get embeddings and accumulate them
            embeddings = model.get_embedding(input_ids, attention_mask)
            negative_centroid += embeddings.sum(dim=0)
            negative_count += embeddings.size(0)

            # Move variables to CPU and clear cache
            del input_ids, attention_mask, embeddings
            torch.cuda.empty_cache()

    # Compute the final centroids
    positive_centroid /= positive_count
    negative_centroid /= negative_count

    return positive_centroid, negative_centroid


In [70]:
import os
import torch

centroids_path = 'centroids.pth'

positive_centroid = None
negative_centroid = None

if os.path.exists(centroids_path):
    # Load the existing centroids
    centroids = torch.load(centroids_path)
    positive_centroid = centroids['positive_centroid']
    negative_centroid = centroids['negative_centroid']
    print("Loaded existing centroids.")
else:
    # Calculate new centroids if the file does not exist
    positive_centroid, negative_centroid = calculate_centroids(model, small_train_data, batch_size=2, device=device)
    print("Calculated new centroids.")

    # Save the centroids
    torch.save({
        'positive_centroid': positive_centroid,
        'negative_centroid': negative_centroid
    }, centroids_path)
    print("Saved new centroids.")

# Centroid shapes
print(f"Positive Centroid Shape: {positive_centroid.shape}")
print(f"Negative Centroid Shape: {negative_centroid.shape}")


Processing negative batches: 100%|██████████| 500/500 [00:15<00:00, 33.31it/s]

Calculated new centroids.
Saved new centroids.
Positive Centroid Shape: torch.Size([768])
Negative Centroid Shape: torch.Size([768])


In [71]:
print(negative_centroid)

tensor([ 4.3672e-02,  3.0095e-01, -6.3735e-01, -3.2780e-01,  6.5407e-02,
        -1.5481e-01,  5.9975e-01, -1.5350e-02, -2.1214e-01,  2.4663e-02,
        -7.1075e-01, -4.1093e-01, -4.1983e-01,  6.7573e-01, -2.9853e-01,
         6.6122e-02,  3.4857e-01,  2.1945e-01, -5.0165e-01,  4.2773e-02,
        -1.7220e-01,  3.2676e-01, -3.3906e-01,  6.3124e-01, -6.6818e-01,
        -1.4165e-01,  3.7444e-01,  4.2667e-01, -6.5341e-01, -2.3434e-01,
         1.3438e-02, -1.5603e-01,  3.6670e-01, -1.5577e-01, -1.7322e-02,
        -4.7269e-02, -7.9037e-01, -1.5919e-01, -2.8011e-01,  2.6997e-01,
        -2.2315e-01, -2.0017e-01,  6.5705e-02,  8.8200e-01,  5.0926e-01,
        -2.4922e-01, -2.2667e+00,  3.4596e-01,  1.3593e-01,  1.4436e-01,
         1.1462e-02,  3.9647e-01,  4.2943e-01,  6.5910e-01,  7.8862e-01,
         1.7488e-01, -1.8386e-01, -6.8935e-01,  2.3078e-01,  3.6358e-02,
         1.2213e-01,  1.5300e-01, -3.2742e-01, -5.1904e-01, -3.2555e-01,
        -1.3844e-01, -1.3166e-01,  8.2815e-01, -4.6

# Create DistilLBERTCL-Inference Model
Since pretrained DistilBERTCL model accepts pairs of sentences as input, we need DistilBERTCL-Inference that accepts one sentence only for inference

In [72]:
import torch
from transformers import DistilBertModel, DistilBertPreTrainedModel
from transformers.modeling_outputs import SequenceClassifierOutput
from dataclasses import dataclass
from typing import Optional
from alignment.models.distilbert_cl import DistilBertCLModel


class DistilBertCLInferenceModel(DistilBertCLModel):
    def __init__(self, config):
        super().__init__(config)
        # Initialize the centroids with the correct dimensions (768)
        self.positive_centroid = torch.zeros(768).cuda()  # Move to GPU
        self.negative_centroid = torch.zeros(768).cuda()

    def forward(self, input_ids=None, attention_mask=None, return_dict=True):
        # Single sentence input here, no pairs.
        # print("\nDEBUG - DistilBertCLInferenceModel forward pass")

        # Get embedding for the input sentence
        sent_embed = self.get_embedding(input_ids, attention_mask)

        # Ensure the centroids and embeddings are on the same device
        device = sent_embed.device
        self.positive_centroid = self.positive_centroid.to(device)  # Move positive centroid to the same device
        self.negative_centroid = self.negative_centroid.to(device)  # Move negative centroid to the same device

        # Return logits directly
        logits = self.classify_sentiment(sent_embed)

        if not return_dict:
            return logits

        return SequenceClassifierOutput(
            logits=logits
        )

    def classify_sentiment(self, embedding):
        # Set during initialization or externally
        positive_centroid = self.positive_centroid
        negative_centroid = self.negative_centroid


        # Compute cosine similarity between input embedding and centroids
        positive_similarity = torch.cosine_similarity(embedding, positive_centroid)
        negative_similarity = torch.cosine_similarity(embedding, negative_centroid)

        # Get the predicted class based on comparing the similarities (higher similarity -> closer to centroid)
        # print(f"Positive similarity: {positive_similarity.item()}")
        # print(f"Negative similarity: {negative_similarity.item()}")

        # Return a tensor with both similarities as logits for pipeline compatibility
        return torch.tensor([negative_similarity, positive_similarity]).unsqueeze(0).to(embedding.device)

    def set_centroids(self, positive_centroid, negative_centroid):
        self.positive_centroid = positive_centroid
        self.negative_centroid = negative_centroid


In [73]:
from transformers import AutoModelForSequenceClassification

inference_model = DistilBertCLInferenceModel.from_pretrained(model_name)
inference_model.eval()

# Set the centroids in the inference model
inference_model.set_centroids(positive_centroid, negative_centroid)

# Sample prediction with custom text

In [74]:
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          pipeline)

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Load model and tokenizer using the pipeline
pipe = pipeline(task="text-classification", model=inference_model, tokenizer=tokenizer, framework="pt", device=0)

# Sample texts for inference
texts = [
    "The movie was absolutely fantastic!",
    "I didn't enjoy the film; it was quite boring.",
    "An average experience, nothing special or memorable."
]

# Perform inference
# LABEL_0 for negative, LABEL_1 for positive
results = pipe(texts)
for text, result in zip(texts, results):
    print(f"Input: {text}")
    print(f"Output: {result}\n")

The model 'DistilBertCLInferenceModel' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSequenceClassification', 'Gemma

Input: The movie was absolutely fantastic!
Output: {'label': 'LABEL_1', 'score': 0.7010682225227356}

Input: I didn't enjoy the film; it was quite boring.
Output: {'label': 'LABEL_0', 'score': 0.7174751162528992}

Input: An average experience, nothing special or memorable.
Output: {'label': 'LABEL_0', 'score': 0.6645347476005554}



# Prediction with test data

In [75]:
from transformers import pipeline
from tqdm import tqdm

def predict(test, model, adapter, batch_size=16):
    y_pred = []

    pipe = pipeline(task="text-classification", model=model, tokenizer=tokenizer, framework="pt", device=0)

    for i in tqdm(range(0, len(test), batch_size)):
        batch = test[i:i+batch_size]
        results = pipe(batch, truncation=True)

        for result in results:
            standardized_label = adapter(result)
            y_pred.append(standardized_label)

    return y_pred


In [76]:
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from collections import Counter

def evaluate_binary(y_true, y_pred):
    # Use binary mapping
    mapping = {'positive': 1, 'negative': 0}

    # Convert y_true and y_pred based on binary mapping
    y_true_binary = [0 if label == 0 else 1 for label in y_true]  # IMDB labels are 0 and 1
    y_pred_binary = [mapping[label] if label in mapping else 0 for label in y_pred]

    # Calculate overall accuracy
    accuracy = accuracy_score(y_true=y_true_binary, y_pred=y_pred_binary)
    print(f"Binary Accuracy: {accuracy:.3f}")

    print("\nBinary Classification Report:")
    print(classification_report(y_true=y_true_binary, y_pred=y_pred_binary, target_names=['negative', 'positive']))

    print("\nBinary Confusion Matrix:")
    print(confusion_matrix(y_true=y_true_binary, y_pred=y_pred_binary))




In [77]:
# Adapter for ('LABEL_0'/'LABEL_1')
def adapter_model_numeric(output):
    if output['label'] == 'LABEL_1':
        return 'positive'
    elif output['label'] == 'LABEL_0':
        return 'negative'
    else:
        return 'none'

# Adapter for ('POSITIVE'/'NEGATIVE'/'NEUTRAL')
def adapter_model_text(output):
    label = output['label'].lower()
    if "positive" in label:
        return 'positive'
    elif "negative" in label:
        return 'negative'
    elif "neutral" in label:
        return 'neutral'
    else:
        return 'none'

# Evaluation

In [78]:
y_pred = predict(X_test, inference_model, adapter_model_numeric, batch_size=16)

The model 'DistilBertCLInferenceModel' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSequenceClassification', 'Gemma

In [81]:
# Evaluate function for binary classification
evaluate_binary(y_true, y_pred)

Binary Accuracy: 0.896

Binary Classification Report:
              precision    recall  f1-score   support

    negative       0.95      0.83      0.89     12500
    positive       0.85      0.96      0.90     12500

    accuracy                           0.90     25000
   macro avg       0.90      0.90      0.90     25000
weighted avg       0.90      0.90      0.90     25000


Binary Confusion Matrix:
[[10426  2074]
 [  536 11964]]


In [80]:
# Print samples of predictions and true labels for verification
print("Sample y_pred:", y_pred[:100])
print("Sample y_true:", y_true[:100])

Sample y_pred: ['negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 